In [1]:
#import os
#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512'

!git clone https://github.com/federico2879/MLDL2024_semantic_segmentation.git

from MLDL2024_semantic_segmentation.datasets.importDataset import Download
from MLDL2024_semantic_segmentation.datasets.importDataset import Modified_CityScapes
import torch
from torch import nn
from torch.utils.data import DataLoader
import torchvision
from MLDL2024_semantic_segmentation.datasets.cityscapes import CityScapes
import torchvision.transforms as transforms
from MLDL2024_semantic_segmentation.models.deeplabv2.deeplabv2 import *





Cloning into 'MLDL2024_semantic_segmentation'...
remote: Enumerating objects: 216, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 216 (delta 53), reused 1 (delta 1), pack-reused 114
Receiving objects: 100% (216/216), 141.56 KiB | 2.78 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [2]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [23]:
!pip install fvcore

from fvcore.nn import FlopCountAnalysis, flop_count_table
import time
import numpy as np

def Flops(model, height, width):
  image = torch.zeros((3, height, width))
  flops = FlopCountAnalysis(model, image)
  flops_CT = flop_count_table(flops)
  print(flops_CT)
  return flops, flops_CT
'''
def Latency_FPS(model, height, width):
  image = torch.rand((3, height, width))
  iterations = 1000
  latency = []
  FPS = []

  for i in range(iterations):
    start = time.time()
    output = model(image)
    end = time.time()
    ltc_i = end-start
    latency.append(ltc_i)
    FPS_i = 1/ltc_i
    FPS.append(FPS_i)

  meanLatency = mean(latency)*1000
  stdLatency = mstd(latency)*1000
  meanFPS = mean(FPS)*1000
  stdFPS = mstd(latency)*1000
  return meanLatency, stdLatency, meanFPS, stdFPS
'''
def fast_hist(a, b, n):
    '''
    a and b are predict and mask respectively
    n is the number of classes
    '''


    k = (a >= 0) & (a < n)
    return np.bincount(n * a[k].astype(int) + b[k], minlength=n ** 2).reshape(n, n)


def per_class_iou(hist):
    epsilon = 1e-5
    return (np.diag(hist)) / (hist.sum(1) + hist.sum(0) - np.diag(hist) + epsilon)

def meanIOU(num_classes, pred, target):
  mIOU = 0
  for i in range(len(pred)):
      hist = fast_hist(pred[i].to("cpu").numpy(), target[i].to("cpu").numpy(), num_classes)
      IOU = per_class_iou(hist)
      mIOU = mIOU + sum(IOU)/num_classes
  return mIOU #*100/len(pred)




In [25]:
# TODO: Define here your training and validation loops.
import torch
import torchvision

def train(model, optimizer, train_loader, loss_fn):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(train_loader):


        #print(f'batch id: {batch_idx}')
        #print(f'(inputs, targets): {(inputs.size(), targets.size())}')
        first_image = inputs[0]

        # Stampiamo le dimensioni della prima immagine nel batch
        #print("Dimensioni della prima immagine nel batch:", first_image.size())
        inputs, targets = inputs.cuda(), targets.cuda()

        # Compute prediction and loss
        outputs =  model(inputs)
        '''
       # print(f'outputs[0]: {outputs[0]}')
        print(f'outputs[0] type: {outputs[0].type()}')
        print(f'outputs[0] size: {outputs[0].size()}')


        #print(f'targets: {targets}')
        print(f'targets type: {targets.type()}')
        print(f'targets size: {targets.size()}')
        '''
        #Ridimensioning tensor
        targets = targets.squeeze(dim=1)
        #print(f'targets size: {targets.size()}')

        targets = targets.long()

        loss = loss_fn(outputs[0], targets)

        # Backpropagation
        optimizer.zero_grad() # reset gradients of parameters
        loss.backward()  # backpropagate the prediction loss
        optimizer.step() # update model

        running_loss += loss.item()
        _, predicted = outputs[0].max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100. * correct / total
    return train_accuracy

def test(model, test_loader, loss_fn):
    model.eval()
    test_loss = 0
    correct = 0
    total_images = 0
    total_iou = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(test_loader):
            inputs, targets = inputs.cuda(), targets.cuda()
            outputs = model(inputs)
            #Ridimensioning tensor+
            '''
            print(f'outputs: {outputs}')
            print(f'outputs type: {outputs.type()}')
            print(f'outputs size: {outputs.size()}')


            print(f'outputs[0]: {outputs[0]}')

            print(f'outputs[0] type: {outputs[0].type()}')
            print(f'outputs[0] size: {outputs[0].size()}')


            #pri nt(f'targets: {targets}')
            print(f'targets type: {targets.type()}')
            print(f'targets size: {targets.size()}')
            '''
            targets = targets.squeeze(dim=1)

            #print(f'targets size: {targets.size()}')

            targets = targets.long()
            #print(f'targets type: {targets.type()}')
            #print(f'targets size: {targets.size()}')
            loss = loss_fn(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            print(f'predicted: {predicted}')
            iou = meanIOU(outputs.size()[1], predicted, targets) #sum of meanIOU over classes di tutte le immagini nel batch
            #total += targets.size(0)
            #correct += predicted.eq(targets).sum().item()
            total_iou += iou.sum().item()  #somma di tytte le singole iou calcolate in precedenza

            print(f'len di targets (=batch_size?): {len(targets)}')
            total_images += len(targets)

    result= total_iou/total_images
    #test_loss = test_loss / len(test_loader)
    #test_accuracy = 100. * correct / total
    return result

In [4]:
# Take dataset
Download('drive/MyDrive/Colab Notebooks/Cityscapes.zip', '')
Modified_CityScapes('Cityscapes/Cityspaces')

Mounted at /content/drive
The zip file has been extracted correctly


In [5]:
# Setup fixed parameters
num_epochs = 5

In [6]:
# Transformations
transform_image = transforms.Compose([
    transforms.Resize((512, 256)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])
transform_target = transforms.Compose([
    transforms.Resize((512, 256)),
    transforms.ToTensor()
])

# Create dataloader
dataset_train = CityScapes('Cityscapes/Cityspaces', split = 'train', transform = transform_image, label_transform = transform_target)#transform_target)
dataloader_train = DataLoader(dataset_train, batch_size=4, shuffle=True)

#print(dataset_train.__getitem__(0)[1].size())
# Prendi il primo batch di dati dal DataLoader
first_batch_inputs, first_batch_targets = next(iter(dataloader_train))

# Calcola il numero di immagini nel primo batch
batch_size = first_batch_inputs.size(0)
batch_size2 = first_batch_targets.size(0)

print("Dimensione del batch inputs nel DataLoader:", batch_size)
print("Dimensione del batch targets nel DataLoader:", batch_size2)

dataset_val = CityScapes('Cityscapes/Cityspaces', split = 'val', transform = transform_image, label_transform = transform_target)#transform_target)
dataloader_val = DataLoader(dataset_val, batch_size=4, shuffle=False)


Dimensione del batch inputs nel DataLoader: 4
Dimensione del batch targets nel DataLoader: 4


In [7]:


#os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'max_split_size_mb:512, 256'
#torch.cuda.empty_cache()

In [8]:
# Pretrained
pretrained = torchvision.models.resnet101(pretrained=True)
torch.save(pretrained.state_dict(), "DeepLab_resnet_pretrained_imagenet.pth")

# Inizialization of the model
model = get_deeplab_v2().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet101-63fe2227.pth" to /root/.cache/torch/hub/checkpoints/resnet101-63fe2227.pth
100%|██████████| 171M/171M [00:01<00:00, 135MB/s]


Deeplab pretraining loading...


In [9]:
# Define loss and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [10]:
# Set the random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

for epoch in range(num_epochs):
    print(f'Epoch number: {epoch}')
    train(model, optimizer, dataloader_train, loss_fn)
    test_acc = test(model, dataloader_val, loss_fn)
    print(f"Test accuracy: {test_acc:.2f}")

Epoch number: 0
Test accuracy: 12216209.80
Epoch number: 1


KeyboardInterrupt: 

In [26]:
test_acc2 = test(model, dataloader_val, loss_fn)



len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di targets (=batch_size?): 4
len di tar

In [27]:
print(test_acc2)

0.07684691662747435
